In [11]:
%pip install pandas
%pip install telethon 
%pip install Pillow 
%pip install python-magic-bin numpy
import os
import re
import pandas as pd
from datetime import datetime
%pip install telethon
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto, MessageMediaDocument
import numpy as np
from PIL import Image
import io
import magic
from collections import defaultdict

# Configuration
API_ID = '20456758'  # Replace with your API ID
API_HASH = '83bb06a8c677ed8128784c2dc575aff6'  # Replace with your API hash
PHONE_NUMBER = '+251920747086'  # Replace with your phone number
SESSION_NAME = 'amharic_ecommerce_scraper'
# List of Ethiopian e-commerce channels to monitor
CHANNELS = [
    '@ZemenExpress',          
    '@nevacomputer',    
    '@meneshayeofficial',
    '@ethio_brand_collection',       
    '@Leyueqa' 
    '@AwasMart',    
]
# Output directories
RAW_DATA_DIR = 'raw_data'
PROCESSED_DATA_DIR = 'processed_data'
IMAGES_DIR = os.path.join(RAW_DATA_DIR, 'images')
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

class TelegramScraper:
    def __init__(self):
        self.client = TelegramClient(SESSION_NAME, API_ID, API_HASH)
        self.client.start(PHONE_NUMBER)

    def fetch_channel_messages(self, channel, limit=100):
        """
        Fetch messages from a Telegram channel.
        Returns a list of dictionaries representing messages.
        """
        messages = []
        try:
            entity = self.client.get_entity(channel)
            for message in self.client.iter_messages(entity, limit=limit):
                messages.append({
                    'channel': channel,
                    'message_id': message.id,
                    'date': message.date,
                    'text': message.text,
                    'views': getattr(message, 'views', None),
                    'sender_id': getattr(message, 'sender_id', None)
                })
        except Exception as e:
            print(f"Error fetching messages from {channel}: {e}")
        return messages
 
    def clean_amharic_text(self, text):
        """
        Clean and normalize Amharic text
        """
        if not text:
            return ""
# Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        
        # Remove emojis and special characters (keeping Amharic characters)
        # Amharic Unicode range: U+1200 to U+137F
        text = re.sub(r'[^\u1200-\u137F\s.,!?።፣፤፥፦፧፨0-9a-zA-Z]', '', text)
        
        # Normalize whitespace
        text = ' '.join(text.split())
        
        return text.strip()
    
    def extract_entities(self, text):
        """
        Basic entity extraction for Amharic e-commerce text
        """
        entities = {
            'prices': [],
            'products': [],
            'locations': [],
            'contacts': []
        }
        
        if not text:
            return entities
        
        # Extract prices (numbers with currency symbols or words)
        price_patterns = [
            r'(\d+)\s*(ብር|br|birr|BR|ብሮ|ብሬ)',  # Ethiopian Birr
            r'\$\s*(\d+)',                      # US Dollars
            r'(\d+)\s*(ዶላር|dollar)'             # Dollars in Amharic
        ]
        
        for pattern in price_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                if match[0].isdigit():
                    entities['prices'].append(match[0] + ' ' + (match[1] if len(match) > 1 else 'ብር'))
        
        # Extract phone numbers (Ethiopian format)
        phone_matches = re.findall(r'(?:\+251|0)(?:9\d{8}|[1-9]\d{7})', text)
        entities['contacts'].extend(phone_matches)
        
        # Extract locations (common Ethiopian cities/areas)
        locations = ['አዲስ አበባ', 'ባህር �ር', 'ድሬ ዳዋ', 'ጅማ', 'መቀሌ', 
                    'አዋሳ', 'አርባ ምንጭ', 'አዲስ አበባ', 'ባህር ዳር', 'ጎንደር']
        entities['locations'] = [loc for loc in locations if loc in text]
        
        return entities
    
    def download_media(self, message):
        """
        Download and process media files (images, documents)
        """
        media_info = {
            'media_type': None,
            'file_path': None,
            'file_size': None,
            'dimensions': None
        }
        
        if not message.media:
            return media_info
        
        try:
            if isinstance(message.media, MessageMediaPhoto):
                # Download photo
                media_info['media_type'] = 'photo'
                file_path = os.path.join(IMAGES_DIR, f'photo_{message.id}.jpg')
                self.client.download_media(message.media, file=file_path)
                
                # Get image dimensions
                with Image.open(file_path) as img:
                    media_info['dimensions'] = f"{img.width}x{img.height}"
                
                media_info['file_path'] = file_path
                media_info['file_size'] = os.path.getsize(file_path)
                
            elif isinstance(message.media, MessageMediaDocument):
                # Download document
                media_info['media_type'] = 'document'
                file_name = f'document_{message.id}'
                file_path = os.path.join(RAW_DATA_DIR, file_name)
                self.client.download_media(message.media, file=file_path)
                
                # Get file type
                mime = magic.Magic(mime=True)
                file_type = mime.from_file(file_path)
                
                # Rename file with proper extension
                ext = file_type.split('/')[-1]
                new_path = f"{file_path}.{ext}"
                os.rename(file_path, new_path)
                
                media_info['file_path'] = new_path
                media_info['file_size'] = os.path.getsize(new_path)
                
        except Exception as e:
            print(f"Error downloading media: {e}")
        
        return media_info
    
    def fetch_channel_messages(self, channel_name, limit=1000):
        """
        Fetch messages from a Telegram channel
        """
        print(f"Fetching messages from {channel_name}...")
        
        messages_data = []
        try:
            channel = self.client.get_entity(channel_name)
            
            for message in self.client.iter_messages(channel, limit=limit):
                try:
                    # Basic message info
                    msg_data = {
                        'channel': channel_name,
                        'message_id': message.id,
                        'date': message.date,
                        'text': self.clean_amharic_text(message.text),
                        'views': message.views if hasattr(message, 'views') else None,
                        'sender': message.sender_id if hasattr(message, 'sender_id') else None
                    }
                    
                    # Download and process media
                    media_info = self.download_media(message)
                    msg_data.update(media_info)
                    
                    # Extract entities from text
                    entities = self.extract_entities(message.text)
                    msg_data.update(entities)
                    
                    messages_data.append(msg_data)
                    
                except Exception as e:
                    print(f"Error processing message {message.id}: {e}")
                    continue
                
        except Exception as e:
            print(f"Error accessing channel {channel_name}: {e}")
        
        return messages_data
    
    def save_to_csv(self, data, filename):
        """Save data to CSV file"""
        df = pd.DataFrame(data)
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        df.to_csv(filepath, index=False, encoding='utf-8')
        print(f"Saved {len(df)} records to {filepath}")
    
    def run(self):
        """Main execution method"""
        all_messages = []
        
        # Fetch messages from all channels
        for channel in CHANNELS:
            channel_messages = self.fetch_channel_messages(channel)
            all_messages.extend(channel_messages)
            
            # Save individual channel data
            self.save_to_csv(channel_messages, f'{channel}_messages.csv')
        
        # Save combined data
        if all_messages:
            self.save_to_csv(all_messages, 'all_channels_combined.csv')
        
        print("Data collection complete!")

if __name__ == "__main__":
    scraper = TelegramScraper()
    scraper.run()
class DataPreprocessor:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        
    def load_data(self):
        """Load all CSV files from the data directory, skipping empty files"""
        all_data = []
        for file in os.listdir(self.data_dir):
            if file.endswith('.csv'):
                filepath = os.path.join(self.data_dir, file)
                try:
                    # Check if file is not empty before reading
                    if os.path.getsize(filepath) == 0:
                        print(f"Skipped empty file: {filepath}")
                        continue
                    df = pd.read_csv(filepath, encoding='utf-8')
                    if not df.empty:
                        all_data.append(df)
                except pd.errors.EmptyDataError:
                    print(f"Skipped empty file: {filepath}")
        
        if all_data:
            return pd.concat(all_data, ignore_index=True)
        return pd.DataFrame()
    
    def preprocess_text(self, text):
        """
        Advanced text preprocessing for Amharic
        """
        if pd.isna(text):
            return ""
        
        # Remove repeated characters (common in informal text)
        text = re.sub(r'(.)\1{2,}', r'\1', text)
        
        # Normalize Amharic numbers to Western numerals
        amharic_numerals = {
            '፩': '1', '፪': '2', '፫': '3', '፬': '4', '፭': '5',
            '፮': '6', '፯': '7', '፰': '8', '፱': '9', '፲': '10'
        }
        
        for am_num, num in amharic_numerals.items():
            text = text.replace(am_num, num)
        
        return text
    
    def process_data(self, df):
        """
        Process the raw dataframe
        """
        # Check if 'text' column exists
        if 'text' not in df.columns:
            print("Warning: 'text' column not found in DataFrame. Returning empty DataFrame.")
            return pd.DataFrame()
        
        # Clean text
        df['clean_text'] = df['text'].apply(self.preprocess_text)
        
        # Convert date to datetime if 'date' column exists
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
        
        # Extract additional features
        df['text_length'] = df['clean_text'].apply(len)
        df['word_count'] = df['clean_text'].apply(lambda x: len(x.split()))
        
        # Handle missing values for expected columns
        for col in ['prices', 'locations', 'contacts']:
            if col in df.columns:
                df[col] = df[col].fillna('[]')
        
        return df
    
    def save_processed_data(self, df, output_file):
        """Save processed data to file"""
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Processed data saved to {output_file}")
# ...existing code...
if __name__ == "__main__":
    scraper = TelegramScraper()
    scraper.run()
    print("Scraping complete!")
    preprocessor = DataPreprocessor(RAW_DATA_DIR)  # <-- use RAW_DATA_DIR here
    raw_data = preprocessor.load_data()
    processed_data = preprocessor.process_data(raw_data)
    preprocessor.save_processed_data(processed_data, 'fully_processed_data.csv')
    print("Data preprocessing complete!")
    
    import re
from collections import defaultdict

# Load the dataset (assuming we're using the processed data from Task 1)
import os
if os.path.exists('fully_processed_data.csv') and os.path.getsize('fully_processed_data.csv') > 0:
    df = pd.read_csv('fully_processed_data.csv', encoding='utf-8')
else:
    print("Warning: 'fully_processed_data.csv' is empty or does not exist. Creating empty DataFrame.")
    df = pd.DataFrame()

# Select 50 random messages for labeling
sample_messages = df['text'].dropna().sample(50, random_state=42).tolist()

def tokenize_amharic(text):
    """Tokenize Amharic text (simple whitespace tokenizer with some special handling)"""
    # Split on whitespace but keep common punctuation attached to words
    tokens = re.findall(r'\w+[\']?\w*|[\w\.-]+|[\u1200-\u137F]+|[^\w\s]', text)
    return [token for token in tokens if token.strip()]

def label_message(message):
    """Label entities in a single message according to CoNLL format"""
    tokens = tokenize_amharic(message)
    labels = ['O'] * len(tokens)  # Initialize all tokens as 'O'
    
    # Patterns for entity recognition
    price_pattern = re.compile(r'(\d+)\s*(ብር|br|birr|BR|ብሮ|ብሬ|ዶላር|dollar)')
    location_words = {'አዲስ አበባ', 'ባህር ዳር', 'ድሬ ዳዋ', 'ጅማ', 'መቀሌ', 
                     'አዋሳ', 'አርባ ምንጭ', 'ጎንደር', 'ቦሌ', 'ፒያሳ', 
                     'ሰማንያ', 'አቃቂ', 'ለሚ ካምፓ', 'ሰበር', 'መኮንን'}
    
    # Convert tokens back to string for pattern matching
    text_for_matching = ' '.join(tokens)
    
    # Label prices
    for match in price_pattern.finditer(text_for_matching):
        price_start, price_end = match.span()
        price_tokens = []
        current_pos = 0
        for i, token in enumerate(tokens):
            token_start = current_pos
            token_end = token_start + len(token)
            current_pos += len(token) + 1  # +1 for the space
            
            # Check if token is within the matched price
            if not (token_end <= price_start or token_start >= price_end):
                price_tokens.append(i)
        
        if price_tokens:
            labels[price_tokens[0]] = 'B-PRICE'
            for i in price_tokens[1:]:
                labels[i] = 'I-PRICE'
    
    # Label locations (simple approach)
    for i, token in enumerate(tokens):
        if token in location_words:
            labels[i] = 'B-LOC'
    
    # Label products (this would normally require more sophisticated approach)
    # For demo, we'll look for common product indicators
    product_keywords = {
        'ለገበያ': 'B-PRODUCT',
        'ሽያጭ': 'B-PRODUCT', 
        'ይገኛል': 'B-PRODUCT',
        'መሸጥ': 'B-PRODUCT',
        'መግዛት': 'B-PRODUCT'
    }
    
    for i, token in enumerate(tokens):
        if token in product_keywords:
            # Label the surrounding words as product
            start = max(0, i-3)
            end = min(len(tokens), i+4)
            labels[start] = 'B-PRODUCT'
            for j in range(start+1, end):
                labels[j] = 'I-PRODUCT'
    
    return list(zip(tokens, labels))

def save_to_conll(labeled_data, filename):
    """Save labeled data in CoNLL format"""
    with open(filename, 'w', encoding='utf-8') as f:
        for message in labeled_data:
            for token, label in message:
                f.write(f"{token}\t{label}\n")
            f.write("\n")  # Empty line between messages

# Label all sample messages
labeled_messages = [label_message(msg) for msg in sample_messages]

# Save to CONLL file
save_to_conll(labeled_messages, 'labeled_data.conll')
print(f"Successfully labeled {len(labeled_messages)} messages in CoNLL format.")
# Step 1: Install Dependencies
%pip install -q transformers datasets seqeval

# Step 2: Import Libraries
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    TrainingArguments, Trainer, DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report

# Step 3: Load Tokenizer and Model (Choose one)
# Use a valid public model for Amharic or multilingual NER
# For Amharic NER, you can use: "Davlan/xlm-roberta-base-finetuned-ner-afr"
# Or fallback to multilingual: "xlm-roberta-base"
model_checkpoint = "Davlan/xlm-roberta-base-finetuned-ner-afr"

# Ensure required dependencies are installed
%pip install -q transformers sentencepiece protobuf

from transformers import AutoTokenizer, AutoModelForTokenClassification

# Use use_fast=False for compatibility with some multilingual models
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)
#model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=5)  # adjust num_labels as needed

# Step 4: Load or Parse CoNLL-format Data
# using Hugging Face `load_dataset`
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=5)  # adjust num_labels as needed
from transformers import AutoTokenizer, AutoModelForTokenClassification
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)
# Function to parse CoNLL file into a Hugging Face Dataset
from datasets import Dataset

def parse_conll_file(filepath):
    sentences, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as file:
        words, tags = [], []
        for line in file:
            if line.strip() == "":
                if words:
                    sentences.append(words)
                    labels.append(tags)
                    words, tags = [], []
            else:
                parts = line.strip().split()
                words.append(parts[0])
                tags.append(parts[-1])
    return Dataset.from_dict({"tokens": sentences, "ner_tags": labels})

dataset = parse_conll_file("labeled_data.conll")  
label_list = list(set(tag for row in dataset["ner_tags"] for tag in row))
label_list.sort()
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

# Convert labels to ids
def encode_labels(example):
    example["labels"] = [label2id[tag] for tag in example["ner_tags"]]
    return example

dataset = dataset.map(encode_labels)

# Load the tokenizer if not already loaded
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# Step 5: Tokenize and Align Labels
def tokenize_and_align_labels(example):
    tokenized = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(example["labels"][word_idx])
        else:
            label_ids.append(example["labels"][word_idx] if label_list[example["labels"][word_idx]].startswith("B-") else example["labels"][word_idx])
        previous_word_idx = word_idx
    tokenized["labels"] = label_ids
    return tokenized

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)

# Step 6: Split Train and Validation
train_test = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

# Step 7: Set Up Trainer
args = TrainingArguments(
    output_dir="./amharic-ner",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=2)
    labels = pred.label_ids
    true_preds = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    return {"f1": classification_report(true_labels, true_preds, output_dict=True)["weighted avg"]["f1"]}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Step 8: Train Model
trainer.train()

# Step 9: Save Fine-Tuned Model
model.save_pretrained("./amharic-ner-model")
tokenizer.save_pretrained("./amharic-ner-model")
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_metric
import numpy as np
from seqeval.metrics import classification_report

# Models to Compare
models = {
    "xlm-roberta": "Davlan/xlm-roberta-base-ner-hrl",
    "mbert": "bert-base-multilingual-cased",
    "distilbert": "Davlan/distilbert-base-multilingual-cased-ner-hrl"
}

def train_and_evaluate(model_name, model_checkpoint, train_dataset, eval_dataset, label_list, id2label, label2id):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id)

    def tokenize_and_align_labels(example):
        tokenized = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
        word_ids = tokenized.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(example["labels"][word_idx])
            else:
                label_ids.append(example["labels"][word_idx] if label_list[example["labels"][word_idx]].startswith("B-") else example["labels"][word_idx])
            previous_word_idx = word_idx
        tokenized["labels"] = label_ids
        return tokenized

    tokenized_train = train_dataset.map(tokenize_and_align_labels)
    tokenized_eval = eval_dataset.map(tokenize_and_align_labels)

    args = TrainingArguments(
        output_dir=f"./results_{model_name}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=10,
    )

    def compute_metrics(pred):
        preds = np.argmax(pred.predictions, axis=2)
        labels = pred.label_ids
        true_preds = [
            [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(preds, labels)
        ]
        true_labels = [
            [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(preds, labels)
        ]
        return {"f1": classification_report(true_labels, true_preds, output_dict=True)["weighted avg"]["f1"]}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    results = trainer.evaluate()
    return model, tokenizer, results

# Example Loop
results_dict = {}
for name, checkpoint in models.items():
    print(f"\nTraining model: {name}")
    model, tokenizer, metrics = train_and_evaluate(name, checkpoint, train_dataset, eval_dataset, label_list, id2label, label2id)
    results_dict[name] = metrics

# Best model based on F1 score
best_model_name = max(results_dict, key=lambda x: results_dict[x]['eval_f1'])
print(f"✅ Best Model: {best_model_name} with F1 = {results_dict[best_model_name]['eval_f1']:.4f}")
# Save the best model
best_model = models[best_model_name]
model.save_pretrained(f"./best_amharic_ner_model_{best_model_name}")
tokenizer.save_pretrained(f"./best_amharic_ner_model_{best_model_name}")
!pip install -q shap lime

import shap
import torch
from transformers import pipeline

# Build a prediction pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Sample Amharic text
example_text = "አዲስ ማርቆስ የዋጋ 600 ብር ጫማ ተሽከርካሪ"

# Use SHAP
explainer = shap.Explainer(ner_pipeline)
shap_values = explainer([example_text])
shap.plots.text(shap_values[0])

# Use LIME (optional — works better for classification)
from lime.lime_text import LimeTextExplainer
lime_explainer = LimeTextExplainer(class_names=label_list)

def predict_proba(texts):
    preds = []
    for text in texts:
        outputs = ner_pipeline(text)
        proba = [0]*len(label_list)
        for o in outputs:
            proba[label2id[o['entity_group']]] += 1
        preds.append([p/len(outputs) for p in proba])
    return np.array(preds)

lime_exp = lime_explainer.explain_instance(example_text, predict_proba, num_features=5)
lime_exp.show_in_notebook()
# Save the SHAP and LIME explanations
shap_values.save("shap_explanations.pkl")
lime_exp.save_to_file("lime_explanations.html")
import pandas as pd
from statistics import mean

# Example vendor data format
# Each row: {"vendor_id": "Vendor1", "text": "...", "views": 120, "timestamp": "2024-06-20"}

def analyze_vendor(posts_df, ner_pipeline):
    scorecard = []

    for vendor_id, group in posts_df.groupby("vendor_id"):
        # Posting frequency
        group['timestamp'] = pd.to_datetime(group['timestamp'])
        weekly_posts = group.set_index('timestamp').resample('W').size().mean()

        # Views and top post
        avg_views = group['views'].mean()
        top_post = group.loc[group['views'].idxmax()]
        top_product = ner_pipeline(top_post['text'])

        # Extract product prices for average
        prices = []
        for _, row in group.iterrows():
            entities = ner_pipeline(row['text'])
            for ent in entities:
                if ent['entity_group'] == "PRICE":
                    try:
                        prices.append(float(ent['word'].replace("ብር", "").replace(",", "").strip()))
                    except:
                        continue
        avg_price = mean(prices) if prices else 0

        # Final lending score (simple version)
        lending_score = (0.5 * avg_views) + (0.5 * weekly_posts)

        scorecard.append({
            "vendor_id": vendor_id,
            "avg_views": avg_views,
            "posting_frequency": weekly_posts,
            "avg_price": avg_price,
            "top_product": top_product,
            "lending_score": lending_score
        })

    return pd.DataFrame(scorecard).sort_values("lending_score", ascending=False)

# Usage
vendor_df = pd.read_csv("telegram_posts.csv") # Load your vendor data
score_df = analyze_vendor(vendor_df, ner_pipeline)
print(score_df.head())


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Fetching messages from @ZemenExpress...
Error accessing channel @ZemenExpress: You must use "async for" if the event loop is running (i.e. you are inside an "async def")
Saved 0 records to processed_data\@ZemenExpress_messages.csv
Fetching messages from @nevacomputer...
Error accessing channel @nevacomputer: You must use "async for" if the event loop is running (i.e. you are inside an "async def")
Saved 0 records to processed_data\@nevacomputer_messages.csv
Fetching messages from @meneshayeofficial...
Error accessing channel @meneshayeofficial: You must use "async for" if the event loop is running (i.e. you are inside an "async def")
Saved 0 records to processed

C:\Users\Yohanan\AppData\Local\Temp\ipykernel_12832\1011795864.py:43: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  self.client.start(PHONE_NUMBER)
C:\Users\Yohanan\AppData\Local\Temp\ipykernel_12832\1011795864.py:228: RuntimeWarning: coroutine 'UserMethods.get_entity' was never awaited
  channel_messages = self.fetch_channel_messages(channel)
C:\Users\Yohanan\AppData\Local\Temp\ipykernel_12832\1011795864.py:228: RuntimeWarning: coroutine 'UserMethods.get_entity' was never awaited
  channel_messages = self.fetch_channel_messages(channel)
C:\Users\Yohanan\AppData\Local\Temp\ipykernel_12832\1011795864.py:228: RuntimeWarning: coroutine 'UserMethods.get_entity' was never awaited
  channel_messages = self.fetch_channel_messages(channel)
C:\Users\Yohanan\AppData\Local\Temp\ipykernel_12832\1011795864.py:228: RuntimeWarning: coroutine 'UserMethods.get_entity' was never awaited
  channel_messages = self.fetch_channel_messages(channel)
C:\Users\Yohanan\AppData\Local\Temp\ipy

Processed data saved to fully_processed_data.csv
Data preprocessing complete!
Successfully labeled 50 messages in CoNLL format.
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Yohanan\Amharic-E-commerce-data-extractor\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\Yohanan\Amharic-E-commerce-data-extractor\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Yohanan\Amharic-E-commerce-data-extractor\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\Yohanan\AppData\Local\Temp\pip-build-env-_yvugmf2\overlay\Lib\si

Note: you may need to restart the kernel to use updated packages.


ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
